# Smart Healthcare Chatbot using Hybrid AI (ML + MiniLM + Rule Engine)

## 1. Introduction

This notebook implements an **intelligent healthcare chatbot** that guides users through
their symptoms in a safe, structured, and explainable way.

The chatbot is designed as a **hybrid AI system**:

- **Machine Learning (TF-IDF + Logistic Regression)** to predict the most likely disease group.
- **Semantic Embeddings (MiniLM)** to retrieve the most similar question–answer pair from the dataset.
- **Rule-based Symptom Engine** to handle:
  - Unknown → Known reasoning with follow-up questions.
  - Sinus-type headache detection (headache + cold).
  - Blood pressure (BP)–specific triage.
  - Severe pain triage and emergency detection.
  - Multi-turn reasoning with memory and progression detection.
  - Emotion acknowledgement and contradiction handling.

The goal is to move from a vague, user-friendly symptom description (**unknown**) to a
more structured understanding (**known**), with clear, human-style explanations.


## Project Overview

This notebook presents a Smart Healthcare Chatbot designed to simulate
human-like medical reasoning for educational purposes.

The system focuses on:
- Understanding user symptoms step-by-step
- Asking relevant follow-up questions
- Combining rule-based logic with learned patterns
- Producing explainable and human-readable responses

This is not a medical diagnosis tool.


# Why This Chatbot Is Different ?

Most chatbots give instant answers.
This chatbot behaves like a cautious healthcare assistant:

1. It does NOT jump to conclusions
2. It asks clarifying questions
3. It handles vague input (unknown → known)
4. It explains its reasoning when asked

The goal is **reasoning quality**, not prediction accuracy.

**Install and import dependencies**

In [1]:
!pip -q install sentence-transformers

import pandas as pd
import numpy as np
import torch
from sentence_transformers import SentenceTransformer, util
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression


**2. Dataset Description**

We use a merged healthcare Q&A dataset stored as:

`/content/merged_health_dataset_3500.csv`

Key columns used:

- `disease` – label/category for each entry  
- `question` – a user-like symptom description  
- `answer` – a medically inspired, simple explanation or suggestion  

The chatbot learns:

1. How different disease groups (e.g., headache, cold_flu, stomach, metabolic_bp) are described.
2. How to map new user inputs into the closest disease bucket and answer pattern.

In [2]:
#1. LOAD DATASET

DATASET_PATH = "/content/merged_health_dataset_3500.csv"  # <--- change if needed

df = pd.read_csv(DATASET_PATH)

# Make sure required columns exist
for col in ["disease", "question", "answer"]:
    if col not in df.columns:
        df[col] = ""

df["disease"] = df["disease"].astype(str).str.strip()
df["question"] = df["question"].astype(str)
df["answer"] = df["answer"].fillna("")

print("✅ Rows:", len(df))
print("✅ Diseases:", sorted(df["disease"].unique()))


✅ Rows: 1374
✅ Diseases: ['blood_pressure', 'cold_flu', 'fever', 'headache', 'menstrual', 'metabolic_bp', 'stomach']


 3. **Machine Learning Model: TF-IDF + Logistic Regression**

We first train a traditional ML pipeline:

1. **TF-IDF Vectorizer** – converts each question into a sparse vector of n-grams.
2. **Logistic Regression** – learns to map questions → disease labels.

### TF-IDF Vectorization
TF-IDF converts symptom text into numerical vectors based on:
- Term frequency (how often words appear)
- Inverse document frequency (how unique words are)

This allows the model to understand **important symptom words**
rather than treating all words equally.

###  Logistic Regression Classifier
Logistic Regression is used to:
- Predict the most likely disease category
- Output probability scores (confidence)

It is chosen because:
- It is interpretable
- It performs well on text classification
- It avoids overfitting for medical-style data


In [3]:
# -----------------------------
#  TRAIN TF-IDF + CLASSIFIER
# -----------------------------
vectorizer = TfidfVectorizer(ngram_range=(1, 2), min_df=2, max_df=0.95)
X = vectorizer.fit_transform(df["question"])
y = df["disease"]

clf = LogisticRegression(max_iter=400, n_jobs=-1)
clf.fit(X, y)

print("✅ Classifier trained on:", clf.classes_)


✅ Classifier trained on: ['blood_pressure' 'cold_flu' 'fever' 'headache' 'menstrual' 'metabolic_bp'
 'stomach']


 **4. Semantic Search with MiniLM**

To make the chatbot more “ChatGPT-like”, we add **sentence embeddings**:

- Use `all-MiniLM-L6-v2` from `sentence-transformers`.
- Encode all questions from the dataset.
- For a new user input, find the most similar question by cosine similarity.

This allows the chatbot to reuse the **closest, most relevant answer** instead of relying only on the classifier.

**Load MiniLM and Compute Embeddings**

In [4]:
#  LOAD MiniLM + EMBEDDINGS
# -----------------------------
device = "cuda" if torch.cuda.is_available() else "cpu"
print("🧠 MiniLM running on:", device)

model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2", device=device)

df_embeddings = model.encode(
    df["question"].tolist(),
    convert_to_tensor=True,
    device=device,
    show_progress_bar=True
)

🧠 MiniLM running on: cpu


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/43 [00:00<?, ?it/s]

 **Rule-engine  Symptom Intelligence**

Rules exist because ML alone is unsafe in healthcare.
 **Symptom Rules, Risk Logic and BP / Emergency Triage**

This section defines the **rule-based layer** that sits on top of the ML model:

- Symptom keyword flags for:
  - `headache`, `cold_flu`, `fever`, `stomach`, `metabolic_bp`, `menstrual`
- Red-flag patterns for emergencies (e.g., chest pain + difficulty breathing)
- BP-priority rules: any text mentioning “BP” or “blood pressure” is mapped to `metabolic_bp`
- Emotion detection (scared, worried, anxious)
- Symptom progression and contradiction detection
- Recovery hints and self-care suggestions per disease bucket

In [5]:
# -----------------------------
# RULE BASE DEFINITIONS
# -----------------------------
SYMPTOM_KEYWORDS = {
    "headache":      ["headache", "head pain", "pressure in my head", "behind my eyes", "temple pain"],
    "cold_flu":      ["cold", "flu", "blocked nose", "runny nose", "sneezing", "sore throat", "cough"],
    "fever":         ["fever", "feverish", "temperature", "high temperature", "hot body", "chills"],
    "stomach":       ["stomach", "tummy", "belly", "abdomen", "bloat", "bloated", "gas", "acidity", "loose motion", "diarrhea", "constipation"],
    "metabolic_bp":  ["bp", "blood pressure", "pressure high", "pressure low", "hypertension", "sugar", "diabetes", "glucose"],
    "menstrual":     ["period", "periods", "menstrual", "menstruation", "cycle", "pms", "cramp", "cramps", "flow", "heavy flow"]
}

RED_FLAGS = [
    "chest pain", "pain in chest", "heart pain",
    "difficulty breathing", "cannot breathe", "can't breathe",
    "shortness of breath",
    "fainted", "loss of consciousness",
    "confusion", "very high fever", "uncontrolled bleeding"
]

NON_CRITICAL_AREAS = ["stomach", "belly", "abdomen", "tummy", "head", "back", "leg", "arm", "shoulder", "knee"]

SEVERITY_WORDS = {
    "severe": 2.0,
    "very strong": 2.0,
    "unbearable": 2.0,
    "worst": 2.0
}


 **Helper Fuctions**

 ## Helper Functions: Symptom Understanding, Safety, and Severity Assessment

This section defines the **core helper functions** that support the chatbot’s reasoning and safety logic.  
These functions do **not generate responses directly**. Instead, they extract structured signals from free-text user input, which are later used by the main chatbot engine.

Importantly, this logic is **not hard-coded to specific answers**.  
All decisions are **data-driven**, keyword-based, and combined dynamically with machine learning predictions and conversation memory.

---

### 1. `extract_flags(text)`

This function identifies **which symptom categories are mentioned** in the user’s message.

- It scans the input text for keywords defined in `SYMPTOM_KEYWORDS`
- Each symptom group (e.g., headache, stomach, fever) is mapped to a boolean flag
- Multiple flags can be active at the same time (multi-symptom support)

This allows the chatbot to reason about **overlapping symptoms**, such as:
- headache + cold → sinus-type headache
- stomach pain + bloating → digestion-related issue

This is **not hard-coded diagnosis logic** — it is a flexible signal extraction step.

---

### 2. `mentions_severe_pain(text)`

This function checks whether the user explicitly describes **high-intensity pain** using natural language.

Examples:
- “severe pain”
- “very bad pain”
- “unbearable pain”

It does **not decide the disease**.
Instead, it raises a **severity signal** that influences:
- whether follow-up questions are asked
- whether safety triage is triggered
- how urgent the final guidance should sound

---

### 3. `detect_contradiction(text)`

This function detects **conversation contradictions**, such as when a user says:
- “it’s gone now”
- “actually not”
- “no longer”

When a contradiction is detected, the chatbot can:
- reset internal assumptions
- avoid compounding incorrect context
- continue reasoning safely from updated information

This improves robustness in **multi-turn conversations**.

---

### 4. `is_immediate_emergency(text)`

This function performs **safety-critical screening**.

It checks for:
- known medical red-flag phrases (defined in `RED_FLAGS`)
- dangerous combinations such as **chest + pain**

If triggered, the chatbot immediately:
- bypasses normal reasoning
- advises urgent medical care

This ensures the system **prioritizes user safety over conversational flow**.

---

### 5. `severity_score(flags, text)`

This function computes an **overall severity level** using a weighted heuristic:

- Baseline score from the number of active symptom flags
- Additional weight from intensity words (e.g., “severe”)
- Final classification into:
  - Mild
  - Moderate
  - Elevated

This severity level affects:
- tone of the response
- urgency of recommendations
- recovery expectations shown to the user

Again, this is **rule-based severity assessment**, not hard-coded outcomes.

---



In [6]:
# -----------------------------
# 5. HELPER FUNCTIONS
# -----------------------------
def extract_flags(text: str):
    t = text.lower()
    flags = {}
    for k, words in SYMPTOM_KEYWORDS.items():
        flags[k] = any(w in t for w in words)
    return flags

def mentions_severe_pain(text: str) -> bool:
    t = text.lower()
    return "severe pain" in t or "very bad pain" in t or "unbearable pain" in t

def detect_contradiction(text: str) -> bool:
    t = text.lower()
    return any(x in t for x in ["actually not", "it's gone now", "no longer", "it went away"])

def is_immediate_emergency(text: str) -> bool:
    t = text.lower()
    if any(flag in t for flag in RED_FLAGS):
        return True
    # chest + pain combo is always serious
    if "chest" in t and "pain" in t:
        return True
    return False

def severity_score(flags: dict, text: str):
    t = text.lower()
    score = 0.0
    # symptom-based baseline
    score += sum(1.0 for v in flags.values() if v)
    # words like "severe", etc.
    for w, val in SEVERITY_WORDS.items():
        if w in t:
            score += val
    if score <= 1:
        return "Mild", score
    elif score <= 3:
        return "Moderate", score
    else:
        return "Elevated", score


In [7]:
# -----------------------------
# 6. DIFFERENTIAL QUESTIONS + TIPS + RECOVERY
# -----------------------------
DIFF_QUESTIONS = {
    "headache": [
        "Did this start after long screen time or studying?",
        "Do you have a cold or blocked nose?",
        "Did you sleep less than usual or skip meals?"
    ],
    "cold_flu": [
        "Do you have sore throat or sneezing?",
        "Did this start after cold weather or AC?",
        "Do you have body pain or tiredness?"
    ],
    "fever": [
        "Since when do you feel feverish?",
        "Do you also have chills or body pain?",
        "Did you take any medicine like paracetamol?"
    ],
    "stomach": [
        "Did this start after a particular meal?",
        "Do you feel bloated, gassy, or is it sharp pain?",
        "Do you have loose motion or constipation?"
    ],
    "metabolic_bp": [
        "Have you checked your BP or sugar recently?",
        "Do you feel dizziness or blurred vision?"
    ],
    "menstrual": [
        "Is your period ongoing or about to start?",
        "Is this stronger than your usual cramps?",
        "Do you also have back pain or heavy flow?"
    ]
}

TIPS = {
    "headache": "• Take short breaks from screens\n• Drink water\n• Rest in a dim, quiet room",
    "cold_flu": "• Sip warm fluids (soup, herbal tea)\n• Do gentle steam inhalation\n• Avoid very cold drinks",
    "fever": "• Drink plenty of fluids\n• Rest more than usual\n• Avoid very heavy or greasy food",
    "stomach": "• Sip warm water slowly\n• Avoid oily/spicy or very heavy meals\n• Stay upright for a while after eating",
    "metabolic_bp": "• Avoid very salty/sugary foods\n• Sit and rest quietly\n• Follow your doctor’s BP/sugar plan if you have one",
    "menstrual": "• Use a warm towel/heating pad on lower abdomen\n• Do light stretching or walking\n• Rest if cramps feel intense"
}

RECOVERY_HINTS = {
    "headache": "Many simple headache or sinus patterns settle in 1–3 days with rest, hydration and reduced screen time.",
    "cold_flu": "Mild cold/flu symptoms usually improve in 3–5 days with rest and fluids, but watch for high fever or breathing issues.",
    "fever": "Mild viral fevers often ease in 2–4 days, but persistent or very high fever needs medical attention.",
    "stomach": "Simple indigestion or bloating commonly improves within a day as long as you avoid triggers and stay hydrated.",
    "metabolic_bp": "Blood pressure or sugar concerns should always be taken seriously; regular follow-up with a doctor is important.",
    "menstrual": "Menstrual cramps often ease as the flow reduces, usually over 1–3 days."
}

### Why this is NOT hard-coded

No fixed responses are embedded here  
No single symptom forces a single diagnosis  
 The same input can produce different outcomes depending on:
- symptom combinations
- conversation history
- ML classification confidence
- embedding similarity
- safety rules

These helper functions form a **structured reasoning layer** that works *together* with:
- Machine Learning (TF-IDF + Logistic Regression)
- Semantic Retrieval (MiniLM embeddings)
- Multi-turn memory and follow-up logic

This design mirrors how **real clinical decision support systems** preprocess patient input before making recommendations.


### Disease Classification & Answer Retrieval

This section combines machine learning predictions with rule-based refinement and semantic retrieval.

- **`map_label_to_bucket()`**  
  Maps dataset disease labels into broader clinical buckets (e.g., headache, stomach, fever).  
  This improves generalization and consistency.

- **`classify_with_conf()`**  
  Uses the trained TF-IDF + Logistic Regression model to predict:
  - Most likely disease label  
  - Confidence score  
  - Corresponding bucket

- **`choose_bucket()`**  
  Refines the final bucket selection by prioritizing:
  - Explicit signals (BP, menstrual, stomach keywords)
  - Symptom flags detected from user input  
  - ML prediction as a fallback

- **`retrieve_answer()`**  
  Uses semantic similarity (MiniLM embeddings) to retrieve the most relevant answer
  from the dataset, restricted to the selected bucket when possible.

This design ensures the chatbot is **not rule-hardcoded**.  
Instead, it combines **ML prediction, semantic similarity, and lightweight medical reasoning**
to produce stable, human-readable outputs.


In [8]:
# -----------------------------
# 7. CLASSIFIER + RETRIEVAL
# -----------------------------
def map_label_to_bucket(label: str) -> str:
    l = label.lower()
    if "head" in l:
        return "headache"
    if "cold" in l or "flu" in l:
        return "cold_flu"
    if "fever" in l:
        return "fever"
    if "stomach" in l:
        return "stomach"
    if "menstru" in l or "period" in l:
        return "menstrual"
    return "metabolic_bp"

df["bucket"] = df["disease"].apply(map_label_to_bucket)

def classify_with_conf(text: str):
    vec = vectorizer.transform([text])
    probs = clf.predict_proba(vec)[0]
    idx = np.argmax(probs)
    label = clf.classes_[idx]
    bucket = map_label_to_bucket(label)
    conf = float(probs[idx])
    return label, bucket, conf

def choose_bucket(flags: dict, fallback: str, text: str) -> str:
    t = text.lower()
    # High priority: BP / sugar
    if "bp" in t or "blood pressure" in t or "sugar" in t or "diabetes" in t:
        return "metabolic_bp"
    # Then menstrual if explicit
    if flags.get("menstrual", False):
        return "menstrual"
    # Then stomach
    if flags.get("stomach", False):
        return "stomach"
    # Then cold/flu
    if flags.get("cold_flu", False):
        return "cold_flu"
    # Then fever
    if flags.get("fever", False):
        return "fever"
    # Then headache
    if flags.get("headache", False):
        return "headache"
    # Then metabolic if flagged
    if flags.get("metabolic_bp", False):
        return "metabolic_bp"
    # Otherwise, fallback from ML
    return fallback

def retrieve_answer(text: str, bucket: str):
    # Use only rows from that bucket if possible
    mask = (df["bucket"] == bucket)
    if mask.sum() == 0:
        sub_df = df
        sub_emb = df_embeddings
    else:
        sub_df = df[mask].reset_index(drop=True)
        sub_emb = df_embeddings[mask]

    q_emb = model.encode([text], convert_to_tensor=True, device=device)
    sims = util.cos_sim(q_emb, sub_emb)[0]
    best_idx = int(torch.argmax(sims).item())
    best_sim = float(sims[best_idx])
    answer = sub_df.iloc[best_idx]["answer"]
    return answer, best_sim

###  Functions for Symptom Understanding and Safety

This section contains reusable helper functions that transform raw user text into structured signals for the chatbot.

- **`extract_flags`**  
  Scans the user’s message for medically relevant symptom keywords and converts free-text input into boolean symptom indicators.  
  This enables the chatbot to reason over *patterns* instead of exact sentences.

- **`mentions_severe_pain`**  
  Detects high-intensity pain phrases (e.g., “severe pain”, “unbearable pain”) to trigger careful triage behavior.

- **`detect_contradiction`**  
  Identifies when a user reverses or cancels a previous symptom (e.g., “it went away”), allowing the conversation state


In [9]:

# 8. STATE + EXPLAINABILITY
# -----------------------------
def new_state():
    return {
        "awaiting_clarification": False,
        "awaiting_pain_location": False,
        "initial_text": "",
        "flags": {k: False for k in SYMPTOM_KEYWORDS.keys()},
        "reason": None
    }

state = new_state()

def format_explanation(r: dict) -> str:
    flags_str = ", ".join([k for k, v in r["flags"].items() if v]) or "none"
    return f"""
🧠 **Model Explanation**
- Final category: **{r['final_bucket']}**
- Severity: **{r['severity']}** (score {r['sev_score']:.2f})
- Overall confidence: **{r['overall_conf']:.2f}**

🔎 **Signals I Used**
- Combined text: *{r['combined_text']}*
- Detected symptom flags: **{flags_str}**
- ML label: **{r['ml_label']}** (conf {r['ml_conf']:.2f})
- Embedding similarity: **{r['emb_sim']:.2f}**
- Sinus override: **{r['sinus_override']}**
- BP override: **{r['bp_override']}**
- Severe pain triage: **{r['severe_override']}**

This is not a medical diagnosis, but an explanation of how I matched your message to patterns in my data.
""".strip()

def emergency_message() -> str:
    return (
        "⚠️ Some of the symptoms you mentioned (like chest/heart/lung pain or serious breathing trouble) can be serious.\n"
        "For your safety, it's better to contact a doctor, urgent care, or emergency services instead of relying on chat."
    )

**Chatbot Engine (Unknown → Known → Advice)**
This is the main brain of the chatbot:


Stage 1: First complaint → ask clarification (unknown → known)

Stage 2: Second message → combine text → disease bucket → answer


Sinus logic: headache + cold → sinus-type headache

BP logic: any “BP / blood pressure” mention → metabolic_bp
Severe pain location triage (chest vs stomach vs body)

Emotion acknowledgement: “I’m scared / worried / anxious”
Progression: “now it’s worse” → update reasoning
“why?” → explainability mode

In [10]:
#9CHATBOT ENGINE

def chatbot(user_text: str) -> str:
    global state
    msg = user_text.strip()

    # 1) Explainability triggers
    if msg.lower() in ["why", "why?", "explain", "how did you get this"]:
        if not state.get("reason"):
            return "I don't have a previous decision to explain yet. Tell me your symptoms first."
        return format_explanation(state["reason"])

    # 2) Immediate emergencies
    if is_immediate_emergency(msg):
        state = new_state()/
        return emergency_message()

    # 3) Greetings
    lower = msg.lower()
    if any(g in lower for g in ["hi", "hello", "hey", "good morning", "good evening"]) and len(msg.split()) <= 4:
        state = new_state()
        return (
            "Hello! I'm your healthcare assistant.\n"
            "Tell me in simple words what you're feeling, for example:\n"
            "- I have a headache\n"
            "- I feel feverish\n"
            "- My stomach is bloated"
        )

    # 4) Thanks → close conversation
    if "thank" in lower:
        state = new_state()
        return "You're welcome. I'm here if you need help again."

    # 5) Contradiction → reset
    if detect_contradiction(msg):
        state = new_state()
        return "Okay, I'll reset everything. Tell me your symptoms again from the beginning."

    # 6) If we are currently waiting for pain location (after 'severe pain')
    if state["awaiting_pain_location"]:
        loc_text = msg.lower()
        combined = (state["initial_text"] + " " + msg).strip()

        # Chest / heart / lungs → emergency
        if any(w in loc_text for w in ["chest", "heart", "lungs", "lung"]):
            state = new_state()
            return emergency_message()

        # Decide bucket based on location or words
        if any(w in loc_text for w in ["stomach", "belly", "abdomen", "tummy"]):
            bucket = "stomach"
        elif any(w in loc_text for w in ["head", "forehead", "temple", "behind my eyes", "behind eyes"]):
            bucket = "headache"
        elif any(w in loc_text for w in ["period", "flow", "menstruation", "cramp", "cramps"]):
            bucket = "menstrual"
        else:
            # fallback to classifier
            _, ml_bucket, _ = classify_with_conf(combined)
            bucket = ml_bucket

        flags = extract_flags(combined)
        state["awaiting_pain_location"] = False
        state["awaiting_clarification"] = True
        state["initial_text"] = combined
        state["flags"] = flags

        qs = DIFF_QUESTIONS.get(bucket, [])
        if qs:
            bullets = "\n".join("- " + q for q in qs)
        else:
            bullets = "- Can you describe a bit more about when it started and how it feels?"

        return f"You mentioned: **{combined}**\n\nLet me understand this step-by-step:\n{bullets}"
        # 7) FIRST MESSAGE → ask follow-up
    if not state["awaiting_clarification"]:
        # Severe pain without clear area → ask location
        if mentions_severe_pain(msg):
            t = msg.lower()
            # If clearly chest area with pain → emergency immediately
            if "chest" in t or "heart" in t or "lungs" in t:
                state = new_state()
                return emergency_message()

            # If clearly menstrual
            if any(w in t for w in ["period", "flow", "menstruation", "cramp", "cramps"]):
                flags = extract_flags(msg)
                bucket = "menstrual"
                state["awaiting_clarification"] = True
                state["awaiting_pain_location"] = False
                state["initial_text"] = msg
                state["flags"] = flags
                qs = DIFF_QUESTIONS["menstrual"]
                bullets = "\n".join("- " + q for q in qs)
                return f"You mentioned: **{msg}**\n\nLet me understand this step-by-step:\n{bullets}"

            # Otherwise, ask where the pain is
            state["awaiting_pain_location"] = True
            state["initial_text"] = msg
            return (
                "You mentioned **severe pain**. To guide you safely, where do you feel it the most?\n"
                "- Chest / heart / lungs\n"
                "- Stomach / abdomen\n"
                "- Head / sinus area\n"
                "- General body / muscles"
            )

        # Normal first complaint
        flags = extract_flags(msg)
        ml_label, ml_bucket, ml_conf = classify_with_conf(msg)
        bucket = choose_bucket(flags, ml_bucket, msg)

        state["awaiting_clarification"] = True
        state["initial_text"] = msg
        state["flags"] = flags

        qs = DIFF_QUESTIONS.get(bucket, [])
        if qs:
            bullets = "\n".join("- " + q for q in qs)
        else:
            bullets = "- Can you describe a bit more about when it started and how it feels?"

        return f"You mentioned: **{msg}**\n\nLet me understand this step-by-step:\n{bullets}"

    # 8) SECOND MESSAGE → combine and give final short A+ output
    combined = (state["initial_text"] + " " + msg).strip()
    flags = extract_flags(combined)

    ml_label, ml_bucket, ml_conf = classify_with_conf(combined)
    bucket = choose_bucket(flags, ml_bucket, combined)

    # Sinus override: headache + cold/flu together
    text_lower = combined.lower()
    sinus_override = flags.get("headache", False) and flags.get("cold_flu", False)
    if sinus_override:
        bucket = "headache"
        base_answer = (
            "Because you described both headache and cold-like symptoms, this behaves like a "
            "**sinus-type headache**, where blocked nose and sinus pressure cause pain around the forehead and eyes."
        )
        sim = 0.0
    else:
        base_answer, sim = retrieve_answer(combined, bucket)

    # BP override
    bp_override = "bp" in text_lower or "blood pressure" in text_lower or "pressure" in text_lower and "head" in text_lower
    severe_override = mentions_severe_pain(combined)

    severity, sev_score = severity_score(flags, combined)
    tips = TIPS.get(bucket, "")
    recovery = RECOVERY_HINTS.get(bucket, "")

    overall_conf = (ml_conf + max(sim, 0.0)) / 2.0

    # Store reasoning for "why?"
    state["reason"] = {
        "combined_text": combined,
        "flags": flags,
        "ml_label": ml_label,
        "ml_bucket": ml_bucket,
        "ml_conf": ml_conf,
        "final_bucket": bucket,
        "emb_sim": sim,
        "severity": severity,
        "sev_score": sev_score,
        "overall_conf": overall_conf,
        "sinus_override": sinus_override,
        "bp_override": bp_override,
        "severe_override": severe_override
    }

    # Reset for next conversation
    state["awaiting_clarification"] = False
    state["initial_text"] = ""

    # Build A+ style short output (not huge, but high quality)
    header_line = f"From what you described, this seems most related to **{bucket}** ({severity} level)."

    if sinus_override:
        meaning = (
            "This behaves like a **sinus-type headache**, where cold or congestion creates pressure around the forehead and eyes."
        )
    elif bucket == "metabolic_bp":
        meaning = (
            "This pattern is connected with blood pressure or sugar changes. It is important to follow your doctor's plan and "
            "take BP/sugar symptoms seriously."
        )
    elif bucket == "menstrual":
        meaning = (
            "These cramps and discomfort are likely linked with your **menstrual cycle**. Many people feel bloating, cramping, "
            "back pain or heavy flow during periods."
        )
    elif bucket == "stomach":
        meaning = (
            "This sounds like a **stomach or digestion** issue, often linked with food choices, timing, stress, or routine changes."
        )
    elif bucket == "cold_flu":
        meaning = (
            "This looks like a **cold/flu-type** pattern with symptoms such as congestion, sore throat, or mild fever."
        )
    elif bucket == "fever":
        meaning = (
            "This behaves like a **fever** pattern, often seen in viral infections where the body temperature and fatigue increase."
        )
    else:  # headache
        meaning = (
            "This behaves like a **headache** pattern, which may relate to stress, screen time, posture, or sleep."
        )

    tips_text = tips if tips else "• Rest, hydrate, and notice if anything specific triggers or worsens the symptoms."
    recovery_text = recovery if recovery else "Recovery time can vary, but if this keeps getting worse, a doctor’s advice is safer."

    reply = f"""{header_line}

**In simple words:**
{meaning}

**What you can try now:**
{tips_text}

**Rough recovery expectation:**
{recovery_text}

This is not a medical diagnosis, but guidance based on patterns in my data.
If symptoms become very strong, feel very unusual, or last many days, it's safer to talk with a doctor or urgent care.

(You can type **"why?"** if you want to see how I reasoned this.)
""".strip()

    return reply


**Chatbot UI **

In [ ]:
# -----------------------------
# 10. INTERACTIVE LOOP
# -----------------------------
print("🤖 Smart Healthcare Chatbot  – type 'exit' to stop.\n")

while True:
    try:
        user_input = input("You: ").strip()
    except KeyboardInterrupt:
        print("\nBot: Session ended. Take care. 🌿")
        break

    if user_input.lower() in ["exit", "quit", "bye"]:
        print("Bot: Take care and stay healthy. 🌿")
        break

    bot_reply = chatbot(user_input)
    print("\nBot:", bot_reply)
    print("-" * 60)


🤖 Smart Healthcare Chatbot  – type 'exit' to stop.


Bot: You mentioned: **i have i high temperature**

Let me understand this step-by-step:
- Since when do you feel feverish?
- Do you also have chills or body pain?
- Did you take any medicine like paracetamol?
------------------------------------------------------------
